In [1]:
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb


from sfc import *

In [2]:
biodata = pd.read_csv('../../data/thrombosis_non_thrombosis_v4.csv', sep="\t")

biodata.drop(['node'], axis=1, inplace=True)
biodata

,dg,bt,cl,burts,pr,auth,areaSAS,areaSES,relSESA,consurf,type
0,2,0.000757,0.196523,1.996053,0.001401,0.046619,91.970392,85.097870,0.939882,-0.840,Thrombosis
1,9,0.037589,0.318782,5.691042,0.002637,0.241715,0.000000,0.000000,0.000000,-0.840,Thrombosis
2,8,0.037337,0.306820,4.443999,0.002535,0.201485,57.444680,62.364731,0.585397,-0.343,Non_thrombosis
3,8,0.000803,0.290992,4.189402,0.003369,0.313858,39.795853,53.654439,0.592598,-0.866,Non_thrombosis
4,4,0.002903,0.221407,2.410380,0.002089,0.055087,92.494078,64.243522,0.709552,-0.081,Thrombosis
...,...,...,...,...,...,...,...,...,...,...,...
414,1,0.000000,0.158765,1.000000,0.001131,0.000055,92.780324,51.624957,1.094793,-0.854,Thrombosis
415,1,0.000000,0.180232,1.000000,0.000703,0.000250,110.996267,68.525570,0.992075,-0.844,Thrombosis
416,2,0.004785,0.207217,1.835975,0.001749,0.003968,194.843373,131.003704,1.087150,1.390,Non_thrombosis
417,1,0.000000,0.180549,1.000000,0.000905,0.000852,95.014203,61.300038,1.003882,-0.378,Non_thrombosis


In [3]:
test_idx = [1,2,415,416]
#train_set = biodata.iloc[~biodata.index.isin(test_idx), 0:(biodata.shape[1]-1)]
train_set = biodata.iloc[~biodata.index.isin(test_idx), :]
#test_set = biodata.iloc[test_idx, 0:(biodata.shape[1]-1)]
test_set = biodata.iloc[test_idx, :]

x_train = train_set.iloc[:, 0:(biodata.shape[1]-1)]
y_train = train_set.iloc[:, (biodata.shape[1]-1)]

x_test = test_set.iloc[:, 0:(biodata.shape[1]-1)]
y_test = test_set.iloc[:, (biodata.shape[1]-1)]

y_test.replace(['Thrombosis', 'Non_thrombosis'],[1, 0], inplace=True)
y_train.replace(['Thrombosis', 'Non_thrombosis'],[1, 0], inplace=True)

test_set.reset_index(drop=True, inplace=True)
train_set.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
x_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)
test_set

,dg,bt,cl,burts,pr,auth,areaSAS,areaSES,relSESA,consurf,type
0,9,0.037589,0.318782,5.691042,0.002637,0.241715,0.000000,0.000000,0.000000,-0.840,1
1,8,0.037337,0.306820,4.443999,0.002535,0.201485,57.444680,62.364731,0.585397,-0.343,0
2,1,0.000000,0.180232,1.000000,0.000703,0.000250,110.996267,68.525570,0.992075,-0.844,1
3,2,0.004785,0.207217,1.835975,0.001749,0.003968,194.843373,131.003704,1.087150,1.390,0


In [11]:
def runXGBoost(x_train, y_train, x_test, y_test, prob=False, metric = 'auc'):   

    d_train = xgb.DMatrix(x_train, y_train)
    d_test = xgb.DMatrix(x_test, y_test)

    params = {'booster': 'gbtree',
        'objective': "binary:logistic",
        'nthread': 4,
        'eval_metric': metric}

    myfit = xgb.train(params = params,
        dtrain = d_train,
        num_boost_round = 100,
        maximize = True)

    return (myfit.predict(d_test))

acc, f1, auc, pred = SynClass(x_train, y_train, x_test, y_test, learner = runXGBoost, scoring = 'auc', classThreshold = 0.5, probability = False)

/home/rios/projects/thrombosis/src/classification/sfc.py:128: RuntimeWarning: invalid value encountered in long_scalars
  acc_pos = tp / (tp + fp)


In [12]:
pred

array([0.38218614, 0.31720081, 0.33943908, 0.21270363])

In [6]:
def RSDT(X_train, y_train, X_test, y_test, prob=False, metric = 'auc'):
    params = {'max_depth':np.arange(1, 3, 1), 
              'criterion':np.array(['gini', 'entropy']),
              'min_samples_leaf':np.arange(1, 3, 1)
             }    
   
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    model = DecisionTreeClassifier()
    model_cv = RandomizedSearchCV(model, params, cv = cv, scoring='f1_macro', n_jobs = -1, n_iter=50)

    model_cv.fit(X_train, y_train)
    
    if prob:
        y_pred = model_cv.predict_proba(X_test)
    else:
        y_pred = model_cv.predict(X_test)
    
    return y_pred


acc, f1, auc, pred = SynClass(x_train, y_train, x_test, y_test, learner = RSDT, scoring = 'auc', classThreshold = 0.5, probability = False)

/home/rios/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 8 is smaller than n_iter=50. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


ValueError: 
All the 240 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/home/rios/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/rios/.local/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 969, in fit
    super().fit(
  File "/home/rios/.local/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 172, in fit
    X, y = self._validate_data(
  File "/home/rios/.local/lib/python3.8/site-packages/sklearn/base.py", line 591, in _validate_data
    X = check_array(X, input_name="X", **check_X_params)
  File "/home/rios/.local/lib/python3.8/site-packages/sklearn/utils/vali

In [4]:
# NÃO RODAR
# teste básico de validação com o código em R
def runR(x_train, y_train, x_test, y_test, prob=False, metric = 'auc'):   

    xgb_R = pd.read_csv("/tmp/output_xgb.csv", sep=",", index_col=[0])
    xgb_R.reset_index(drop=True, inplace=True)
    return xgb_R

acc, f1, auc, pred = SynClass(x_train, y_train, x_test, y_test, learner = runR, scoring = 'auc', classThreshold = 0.5, probability = False)

/home/rios/projects/thrombosis/src/classification/sfc.py:131: RuntimeWarning: invalid value encountered in long_scalars
  acc_neg = tn / (tn + fn)
